** Some pactice for CNN and RNN **

In [1]:
import tensorflow as tf
import numpy as np
from random import shuffle
from tensorflow.contrib import rnn
tf.logging.set_verbosity(tf.logging.INFO)
tf.reset_default_graph()
from IPython.core.display import Image, display
from collections import deque

# Use CNN to train MNIST

### Define estimators functions using `tf.estimator`

In [2]:
def GenerateEstimatorSpec(logits, labels, mode):
    # Generate Predictions
    predictions = {
      "classes": tf.argmax(input = logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    print(tf.argmax(input = logits, axis=1))
    # If during training mode, just return the predictions
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, 
                                          predictions = predictions)
    
    # If during Train or Eval, calculate cross-entropy loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, 
                                                  logits = logits)

    # If during Train, update gradients
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, 
                                          loss = loss, 
                                          train_op = train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels = labels, 
              predictions = predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(
      mode = mode, loss = loss, eval_metric_ops = eval_metric_ops) 

### Define CNN network using `layers`

In [3]:
def cnn_model_fn(features, labels, mode, params = None): #params will be used for hyper params
    
    # features: This is the first item returned from the input_fn passed to train
    # labels: This is the second item returned from the input_fn passed to train
    
    #input layer (reshape from 784 cells)
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1]) 
    #[batch_size, image_width, image_height, channels], -1: dynamic compute
    
    # Convolutonal Layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32, #number of filters
        kernel_size = [5,5], #filter size, or kernel_size = 5
        strides=(1, 1), #move along 2 dirs
        padding = "same", # other options: "valid"
        activation = tf.nn.relu,
        name = "CL_1"
        )
    
    # Note the output dimension: 28 * 28 * 32
    
    # Pooling Layer 1
    pool1 = tf.layers.max_pooling2d(inputs = conv1, 
                                    pool_size=[2, 2], 
                                    strides=2)
    
    # Note the output dimension: 14 * 14 * 32
    
    # Convolutonal Layer 2
    conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 64, #number of filters
        kernel_size = [5,5], #filter size
        padding = "same", # other options: "valid"
        activation = tf.nn.relu,
        name = "CL_2"
    )
    
    # Note the output dimension: 14 * 14 * 64
    
    # Pooling Layer 2
    pool2 = tf.layers.max_pooling2d(inputs = conv2, 
                                    pool_size=[2, 2], 
                                    strides=2)
    
    # Note the output dimension: 7 * 7 * 64
    
    # Flatten layer (same below??)
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64]) # 3136 weights
    pool2_flat = tf.layers.flatten(pool2)
    
    # FC layer (dense layer)
    dense = tf.layers.dense(inputs = pool2_flat, 
                            units=1024, # 3136 --> 1024
                            activation = tf.nn.relu)
    
    # Drop-out layer
    dropout = tf.layers.dropout(inputs=dense, 
                                rate=0.4, 
                    training = mode == tf.estimator.ModeKeys.TRAIN)# only activate during training
    
    # Final layer (10 logits)
    logits = tf.layers.dense(inputs = dropout, units = 10)
    


    Final_EstimatorSpec = GenerateEstimatorSpec(logits, labels, mode)
    return(Final_EstimatorSpec)

### Train data

In [4]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [5]:
train_data[0].shape

(784,)

In [6]:
train_data.shape

(55000, 784)

In [7]:
type(train_data)

numpy.ndarray

In [8]:
train_data[0].dtype

dtype('float32')

In [9]:
train_labels.shape

(55000,)

In [10]:
type(train_labels)

numpy.ndarray

In [11]:
train_labels[0].dtype

dtype('int32')

### train_input_fn

In [12]:
# Returns model input
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data}, # features 
    y=train_labels, # targets
    batch_size = 100, 
    num_epochs = None, # forever, the model will train until the specified number of steps is reached
    shuffle = True)

### mnist_classifier

In [13]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn = cnn_model_fn, # model function type, 
    model_dir = "./model_files",
    params = None)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model_files', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c1567eda0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### logging_hook

In [14]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

### Start training

In [16]:
mnist_classifier.train(
    input_fn = train_input_fn,
    steps = 10) # for testing purpose
    #hooks = [logging_hook])

Tensor("ArgMax_1:0", shape=(100,), dtype=int64)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model_files/model.ckpt-2
INFO:tensorflow:Saving checkpoints for 3 into ./model_files/model.ckpt.
INFO:tensorflow:loss = 2.305, step = 3
INFO:tensorflow:Saving checkpoints for 12 into ./model_files/model.ckpt.
INFO:tensorflow:Loss for final step: 2.30147.


In [17]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data}, #features 
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

Tensor("ArgMax_1:0", shape=(?,), dtype=int64)
INFO:tensorflow:Starting evaluation at 2018-04-30-00:22:46
INFO:tensorflow:Restoring parameters from ./model_files/model.ckpt-12
INFO:tensorflow:Finished evaluation at 2018-04-30-00:22:52
INFO:tensorflow:Saving dict for global step 12: accuracy = 0.1444, global_step = 12, loss = 2.30339
{'accuracy': 0.1444, 'loss': 2.303391, 'global_step': 12}


# Use RNN (`LSTM`) to train `sum` operation

### Define RNN network

In [19]:
def rnn_model_fn(features, labels, mode, params = None): #params will be used for hyper params
    
    num_time_steps = 12;
    num_dims = 1;
    num_hidden_units = 24;
    num_outputs = 13;
    
    input_layer = tf.unstack(features['x'], num_time_steps, num_dims)# (data, time_steps, 1), cut into 10 pieces
    
    lstm_layer = rnn.BasicLSTMCell(num_hidden_units )
    
    outputs, _ = rnn.static_rnn(lstm_layer, input_layer ,dtype = "float32")
    
    logits = tf.layers.dense(inputs = outputs[-1], units = num_outputs)
    
    Final_EstimatorSpec = GenerateEstimatorSpec(logits, labels, mode)
    return(Final_EstimatorSpec)

### Generate synthetic training data

In [20]:
train_input = ['{0:012b}'.format(i) for i in range(2**12)] 
shuffle(train_input)
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append(temp_list)

train_input = np.array(ti, dtype=np.float32)
train_input[5]

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.]], dtype=float32)

In [21]:
train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    train_output.append(count)

train_output = np.array(train_output, dtype=np.int32)

In [22]:
NUM_EXAMPLES = 4000
train_data = train_input[:NUM_EXAMPLES]
train_labels = train_output[:NUM_EXAMPLES]
eval_data = train_input[NUM_EXAMPLES:]
eval_labels = train_output[NUM_EXAMPLES:]

### Build model

In [23]:
# Returns model input
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data}, # features 
    y=train_labels, # targets
    batch_size = 5, 
    num_epochs = None, # forever, the model will train until the specified number of steps is reached
    shuffle = True)

In [24]:
# Create the Estimator
sum_classifier = tf.estimator.Estimator(
    model_fn = rnn_model_fn, # model function type, 
    model_dir = "./model_files_2",
    params = None)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model_files_2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c1602d2b0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
sum_classifier.train(
    input_fn = train_input_fn,
    steps = 10)

Tensor("ArgMax_1:0", shape=(5,), dtype=int64)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model_files_2/model.ckpt-60002
INFO:tensorflow:Saving checkpoints for 60003 into ./model_files_2/model.ckpt.
INFO:tensorflow:loss = 0.799017, step = 60003
INFO:tensorflow:Saving checkpoints for 60012 into ./model_files_2/model.ckpt.
INFO:tensorflow:Loss for final step: 0.878255.


### Evaluation

In [27]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": eval_data}, #features 
    y = eval_labels,
    num_epochs = 1,
    shuffle = False)

eval_results = sum_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

Tensor("ArgMax_1:0", shape=(?,), dtype=int64)
INFO:tensorflow:Starting evaluation at 2018-04-30-00:25:22
INFO:tensorflow:Restoring parameters from ./model_files_2/model.ckpt-60012
INFO:tensorflow:Finished evaluation at 2018-04-30-00:25:22
INFO:tensorflow:Saving dict for global step 60012: accuracy = 0.864583, global_step = 60012, loss = 0.611921
{'accuracy': 0.86458331, 'loss': 0.61192131, 'global_step': 60012}


### Prediction

In [33]:
# Predict the model and print results
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data}, #features 
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [34]:
pred_results = sum_classifier.predict(input_fn=pred_input_fn)

In [35]:
pred_classes = []
for i in pred_results:
    pred_classes.append(i['classes'])
print(pred_classes[:10])

Tensor("ArgMax_1:0", shape=(?,), dtype=int64)
INFO:tensorflow:Restoring parameters from ./model_files_2/model.ckpt-60012
[5, 7, 6, 7, 4, 8, 6, 7, 6, 5]


# Use RNN(LSTM) to train `sum` Operation

A Straight forward example to predict many-to-one problem

how to predict '[0,1,1,0]' to 2 (SUM)

http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow/

In [28]:
import numpy as np
import random
from random import shuffle
from tensorflow.contrib import rnn
import tensorflow as tf
tf.reset_default_graph()

In [29]:
train_input = ['{0:010b}'.format(i) for i in range(2**10)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

train_input[0]

array([[1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1]])

In [30]:
train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*11)
    temp_list[count]=1
    train_output.append(temp_list)

train_output[0] # one-hot

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [31]:
NUM_EXAMPLES = 900
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:]
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES]

In [32]:
data.shape

TensorShape([Dimension(None), Dimension(10), Dimension(1)])

In [33]:
data = tf.placeholder(tf.float32, [None, 10, 1]) # Batch_size, Timestep, dimension of each input
input = tf.unstack(data, 10, 1)# (data, time_steps, 1), cut into 10 pieces
y = tf.placeholder(tf.float32, [None, 11])

num_hidden = 24
lstm_layer = rnn.BasicLSTMCell(num_hidden)
outputs, _ = rnn.static_rnn(lstm_layer, input,dtype = "float32")

out_weights = tf.Variable(tf.random_normal([num_hidden, 11]))
out_bias = tf.Variable(tf.random_normal([11]))
prediction = tf.matmul(outputs[-1], out_weights)+ out_bias

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = prediction, labels = y))
opt = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [36]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

batch_size = 10
no_of_batches = int(len(train_input) / batch_size)
epoch = 100
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        acc, _ = sess.run([loss, opt],feed_dict={data: inp, y: out})
        print(acc)

acc = sess.run(accuracy,{data: test_input, y: test_output})
print(sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]}))
print('Epoch {:2d} acc {:3.1f}%'.format(i + 1, 100 * acc))
sess.close()

2.32753
2.15157
2.62552
2.82298
2.24111
2.26625
2.51503
2.07593
2.71411
2.57563
2.4334
2.83717
2.72781
2.42089
2.76062
2.49135
2.40187
2.70758
2.96221
1.71879
3.06882
2.298
2.4643
2.16956
2.19979
2.68674
2.34822
2.79859
2.29716
2.51278
2.1017
2.61681
2.02916
2.36924
1.66094
2.21126
2.05463
2.09963
1.78295
2.16127
2.43425
2.42025
2.08319
1.63084
2.0329
2.18961
1.99859
2.29447
1.94722
2.12683
1.8546
2.19426
2.02093
2.1639
2.51177
2.2689
2.17798
2.22099
2.36133
2.11176
2.18586
2.14479
1.99544
1.64024
2.13601
1.81131
1.90603
1.91002
2.19119
2.30469
2.15371
2.28818
1.86524
2.28062
1.81041
1.79495
1.78504
2.33526
2.22892
1.93522
1.64266
1.74341
1.76424
1.99519
1.76134
2.08655
1.70757
1.7836
1.5553
1.62053
1.7793
2.09074
1.98055
2.14222
1.74109
2.22123
1.82147
1.58442
1.65439
1.85568
1.89975
2.03671
1.9832
1.48855
2.07966
1.65249
1.84262
1.96147
1.99736
1.43378
2.26523
2.01746
1.52969
1.86434
1.94281
2.44214
1.62952
2.61043
1.88682
1.68968
1.58858
2.18749
1.65863
1.9721
1.51496
1.71211
1.9891

0.758954
0.758308
1.48325
0.730316
0.713128
0.737639
1.22955
0.807552
0.780597
0.774244
0.95916
0.746402
0.719881
0.842259
0.804547
0.841008
0.791823
0.926579
0.740168
0.740441
1.03135
0.922782
0.778827
0.853748
0.961854
0.8766
0.987006
0.667411
0.721572
0.806641
0.938587
0.747863
0.748188
0.816776
0.819266
0.836547
0.727507
1.02481
0.695835
0.61887
0.606179
0.628248
0.964246
1.01537
0.691799
0.635025
0.755483
0.783587
0.697993
0.979479
0.704911
0.626728
0.861171
0.866902
0.838838
0.833408
0.660411
0.930522
0.714026
1.10028
0.652698
0.942701
0.711032
0.788206
0.705504
0.794312
0.548009
0.830959
0.902845
0.71948
0.748167
0.956547
0.79183
0.914073
1.11915
0.625326
0.846441
0.78216
0.599281
0.836061
0.591848
0.655989
0.704402
0.706196
0.794481
0.74108
0.765181
0.812444
0.726839
1.04114
0.727817
0.730192
1.44079
0.68571
0.677682
0.710019
1.17705
0.769048
0.746976
0.728236
0.898636
0.703134
0.693115
0.800748
0.779284
0.810177
0.760536
0.86177
0.713475
0.716087
0.98249
0.887475
0.745237
0.80

0.402262
0.530867
0.357171
0.468258
0.555495
0.436451
0.394261
0.626032
0.720829
0.535605
0.877503
0.282182
0.516061
0.470797
0.354155
0.596712
0.385016
0.39666
0.47002
0.445425
0.572594
0.439877
0.39528
0.567171
0.429047
0.673668
0.419024
0.454675
0.891982
0.432826
0.429035
0.422579
0.711669
0.484837
0.458256
0.375791
0.523392
0.368745
0.524218
0.493315
0.495071
0.484615
0.427777
0.391346
0.486381
0.463734
0.582019
0.619607
0.50065
0.396448
0.534258
0.457399
0.557872
0.338464
0.429669
0.405198
0.569322
0.41727
0.421916
0.349974
0.517659
0.468224
0.334556
0.695726
0.413618
0.356511
0.37221
0.370193
0.528213
0.653003
0.357669
0.431255
0.430908
0.415015
0.434128
0.510944
0.411096
0.320914
0.583242
0.493912
0.482485
0.527736
0.305478
0.541105
0.318796
0.676061
0.36002
0.550964
0.419216
0.389199
0.378074
0.503955
0.33874
0.445241
0.527504
0.414742
0.367143
0.598247
0.714854
0.51164
0.853674
0.262294
0.484966
0.449246
0.337412
0.574821
0.365793
0.380487
0.448464
0.426063
0.539129
0.420146
0

0.159399
0.348313
0.283253
0.185289
0.488377
0.278222
0.204958
0.231424
0.230344
0.285342
0.411845
0.187011
0.277558
0.281241
0.219795
0.239364
0.295583
0.225329
0.164334
0.404597
0.295077
0.270873
0.321174
0.17798
0.335968
0.174201
0.430615
0.232241
0.312297
0.240184
0.227584
0.200761
0.309859
0.18903
0.286759
0.314924
0.246231
0.184958
0.394275
0.588809
0.333229
0.672232
0.129723
0.254497
0.2809
0.198016
0.361112
0.211154
0.258697
0.312291
0.274204
0.278562
0.264681
0.218037
0.352979
0.263738
0.412985
0.231562
0.270513
0.643445
0.254525
0.261165
0.201718
0.430757
0.287257
0.245338
0.192351
0.306763
0.177397
0.355469
0.272713
0.243115
0.292382
0.214899
0.175218
0.306922
0.303536
0.359813
0.393342
0.245219
0.220418
0.250712
0.231667
0.344507
0.19059
0.243274
0.186643
0.365537
0.227025
0.227453
0.148362
0.334999
0.268139
0.175097
0.474128
0.267445
0.194619
0.220229
0.220767
0.268378
0.391034
0.176128
0.264751
0.270084
0.206133
0.225093
0.281675
0.212069
0.154309
0.386835
0.283066
0.2537

0.132037
0.159017
0.14686
0.226169
0.101711
0.164792
0.182803
0.110667
0.126055
0.178027
0.116039
0.0884664
0.245572
0.198783
0.129084
0.176871
0.0949005
0.222379
0.100409
0.319989
0.150119
0.171775
0.124087
0.129582
0.10053
0.205346
0.0961187
0.179811
0.205189
0.153224
0.100002
0.273715
0.430593
0.190455
0.562954
0.0613207
0.12273
0.161614
0.110023
0.203042
0.117461
0.167207
0.240665
0.164863
0.13456
0.146632
0.128288
0.204384
0.1655
0.268454
0.154972
0.16862
0.49242
0.158171
0.160748
0.104467
0.291273
0.183529
0.120365
0.104172
0.215994
0.0879913
0.24055
0.145787
0.134982
0.187565
0.110812
0.0858407
0.185846
0.215177
0.245229
0.242832
0.124768
0.135532
0.141077
0.141731
0.210559
0.117197
0.16537
0.0859473
0.223847
0.135316
0.132582
0.0733811
0.230094
0.150382
0.101254
0.356848
0.170175
0.110312
0.12558
0.154957
0.138464
0.214337
0.0964618
0.15714
0.176288
0.104288
0.119355
0.170204
0.10936
0.0840723
0.235286
0.192835
0.120553
0.167866
0.0897677
0.215113
0.0954415
0.312611
0.144557
0.

0.0767766
0.127933
0.0474446
0.142394
0.0861373
0.0871202
0.0434075
0.172808
0.0920196
0.0639986
0.279142
0.112213
0.0698885
0.0780386
0.120707
0.0787465
0.1281
0.056943
0.0985761
0.124559
0.05978
0.0704541
0.108172
0.0612738
0.051749
0.157635
0.145717
0.0613134
0.100887
0.0534762
0.154067
0.0565093
0.248258
0.0978838
0.101079
0.0664134
0.0728344
0.0505024
0.148889
0.0506559
0.113044
0.148315
0.105239
0.0630444
0.211412
0.304563
0.112011
0.47839
0.0311921
0.0603436
0.0949802
0.0610968
0.122048
0.0676239
0.106672
0.181081
0.0997465
0.068118
0.0772939
0.0832895
0.12019
0.104956
0.177325
0.107203
0.112318
0.414461
0.112708
0.0988729
0.0555907
0.199924
0.123447
0.0591456
0.0579688
0.176321
0.0475886
0.166536
0.0807593
0.0830154
0.123232
0.0594547
0.0462792
0.10973
0.156738
0.171335
0.157763
0.068273
0.0832291
0.0886807
0.0929977
0.125417
0.0734595
0.124829
0.0449962
0.135924
0.0823655
0.0838251
0.0412971
0.168014
0.0877031
0.0611078
0.272164
0.107604
0.0669938
0.0745859
0.117725
0.074627
0

KeyboardInterrupt: 